In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import time
import numpy as np

class logistic_regression():
    
    def __init__(self, 
                 lr_rate=0.001, 
                 alpha=0.001, 
                 n_iter_no_change=5, 
                 max_iter=1000, 
                 tol=1e-9):
        
        self.lr_rate = lr_rate # eta (constant that multiplies the weight update term in update function)
        self.alpha = alpha #lambda (constant that multiplies the regularization term in optimization function)
        self.n_iter_no_change = n_iter_no_change
        self.max_iter = max_iter
        self.tol = tol
        self._early_stoing = False
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-(x@self.w+self.b)))
        
    def log_loss(self, x, y):
        log_loss, infi = 0, 1e-90
        for x1, y1 in zip(x, y):
            prob_pos_class = self.sigmoid(x1)
            # adding 1e-90 to avoid log(0) math error
            log_loss += y1*np.log(prob_pos_class+infi) + (1-y1)*np.log(1-prob_pos_class+infi)
        return -log_loss/x.shape[0]
        
    def initializations(self, 
                        x_tr, 
                        y_tr, 
                        x_cv, 
                        y_cv):
        self.wait = 0
        self.n = x_tr.shape[0]
        self.x_tr, self.y_tr = x_tr, y_tr
        self.x_cv, self.y_cv = x_cv, y_cv
        self.w, self.b = self.np.zeros_like(x_tr[0]), 0
        self.tr_log_loss_lst = [self.log_loss(self.x_tr, self.y_tr)]
        self.cv_log_loss_lst = [self.log_loss(self.x_cv, self.y_cv)]
        
    def evaluate_the_model(self, epoch):
        self.tr_log_loss_lst.append(self.log_loss(self.x_tr, self.y_tr))
        self.cv_log_loss_lst.append(self.log_loss(self.x_cv, self.y_cv))
        print(f"---> Epoch {epoch}")
        print(f"Tr ave loss {self.tr_log_loss_lst[-1]} & CV ave loss {self.cv_log_loss_lst[-1]}.")
        print(f"Total training time: {np.round(time.time()-self.now, 3)} seconds.")
        
    def early_stopping(self, epoch):
        if (self.tr_log_loss_lst[-2] - self.tr_log_loss_lst[-1]) <= self.tol:
            if self.wait == self.n_iter_no_change:
                temp = np.round(time.time()-self.now, 3)
                print(f"Convergence after {epoch} epochs time took: {temp} seconds.")
                self._early_stoing = True
            self.wait += 1
        else:
            self.wait = 0
        
    def fit(self, 
            x_tr, 
            y_tr, 
            x_cv, 
            y_cv):
        
        self.initializations(x_tr, 
                             y_tr, 
                             x_cv, 
                             y_cv)
        self.now = time.time()
        for epoch in range(1, self.max_iter+1):
            if self._early_stoing == False:
                for x, y in zip(self.x_tr, self.y_tr):
                    common = self.alpha*(y-self.sigmoid(x))
                    self.w = (1-self.lr_rate*self.alpha/self.n)*self.w + common*x
                    self.b += common
                self.evaluate_the_model(epoch)
                self.early_stopping(epoch)
            else: break
    
    def predict_class(self, x):
        return np.array([1 if (self.sigmoid(x1)>0.5) else 0 for x1 in x])
    
    def plot_log_loss_per_epoch(self, ):
        plt.figure(figsize=(18, 8))
        plt.suptitle('Ploting the log_loss VS Epochs with our Custom SGD Results', fontsize=30)
        
        plt.plot(range(1, len(self.tr_log_loss_lst)+1), 
                 self.tr_log_loss_lst, 
                 marker='o', 
                 label='Change in the tr_log_loss per each epoch')
        plt.plot(range(1, len(self.cv_log_loss_lst)+1), 
                 self.cv_log_loss_lst, 
                 marker='o', 
                 label='Change in the te_log_loss per each epoch')
        
        plt.xticks(range(1, len(self.tr_log_loss_lst)+1), fontsize=15)
        plt.yticks(fontsize=15)
        plt.xlabel('Number of Epochs', fontsize=15)
        plt.ylabel('log_loss Value', fontsize=15)
        plt.legend(fontsize=15)
        
        plt.grid()
        plt.show()

In [17]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=50000, 
                           n_features=15, 
                           n_informative=10, 
                           n_redundant=5,
                           n_classes=2, 
                           weights=[0.7], 
                           class_sep=0.7, 
                           random_state=15)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size=0.2)

In [18]:
model = logistic_regression()
model.fit(X_train, y_train, X_test, y_test)

AttributeError: 'logistic_regression' object has no attribute 'np'

In [21]:
from tensorflow import keras
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name) self.hidden1 = keras.layers.Dense(units, activation=activation) self.hidden2 = keras.layers.Dense(units, activation=activation) self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2]) 
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
model = WideAndDeepModel()

2024-03-30 10:47:16.953591: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-30 10:47:16.953619: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-30 10:47:16.953627: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-30 10:47:16.953703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-30 10:47:16.953750: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
 model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation="softmax")
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (Batch  (None, 784)               3136      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Bat  (None, 100)               4

In [26]:
#gradient clipping 
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)